In [3]:
# Install imutils
!pip install imutils


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25855 sha256=9fd562694556491fd93ea966be75f7c06f6b17c989a269faa84e7e1638f8a862
  Stored in directory: c:\users\biraj\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import cv2
import time
import math
import imutils
import numpy as np
import sys
import argparse

In [19]:
def getAngle(a, b, c):
    angle = math.degrees(math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0]))
    angle = angle + 360 if angle < 0 else angle
    return angle if angle < 180 else 360 - angle

# Check if the code is running in a Jupyter Notebook
if 'ipykernel' in sys.modules:
    args = argparse.Namespace(video='video1.mp4')
else:
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", default='video1.mp4')
    args = parser.parse_args()

protoFile = "pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "pose/coco/pose_iter_440000.caffemodel"
nPoints = 18
POSE_PAIRS = [
    [1, 0], [1, 2], [1, 5], [2, 3], [3, 4], [5, 6], [6, 7], [5, 8], [8, 9], [9, 10],
    [2, 11], [8, 11], [5, 11], [2, 8], [11, 12], [12, 13], [0, 14], [0, 15], [14, 16], [15, 17]
]

inWidth = 368
inHeight = 368
threshold = 0.1
frame_count = 0
total_time = 0

if args.video == "camera":
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(args.video)

# Check if the video capture object was successfully created
if not cap.isOpened():
    print(f"Error: Could not open video source {args.video}")
    sys.exit()

hasFrame, frame = cap.read()

# Check if a frame was successfully read
if not hasFrame:
    print(f"Error: Could not read frame from video source {args.video}")
    cap.release()
    sys.exit()

orig_video_writer = cv2.VideoWriter(
    'output' + str(int(time.time())) + '.avi',
    cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'),
    10, (frame.shape[1], frame.shape[0])
)
stick_diag_video_writer = cv2.VideoWriter(
    'stick_diag' + str(int(time.time())) + '.avi',
    cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'),
    10, (frame.shape[1], frame.shape[0])
)

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)

while cv2.waitKey(1) < 0:
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv2.waitKey()
        break
    
    stick_diag = np.ones(frame.shape, dtype='uint8') * 0
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()
    
    Height = output.shape[2]
    Width = output.shape[3]
    key_points = []
    
    for i in range(nPoints):
        probabilityMap = output[0, i, :, :]
        minVal, prob, minLoc, point = cv2.minMaxLoc(probabilityMap)
        x = (frameWidth * point[0]) / Width
        y = (frameHeight * point[1]) / Height
        
        if prob > threshold:
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            key_points.append((int(x), int(y)))
        else:
            key_points.append(None)
    
    if not (None in [key_points[8], key_points[9], key_points[10]]):
        angle1 = getAngle(key_points[8], key_points[9], key_points[10])
        print("Angle1 : ", angle1)
    else:
        angle1 = -1
    
    if not (None in [key_points[13], key_points[12], key_points[11]]):
        angle2 = getAngle(key_points[13], key_points[12], key_points[11])
        print("Angle2 : ", angle2)
    else:
        angle2 = -1
    
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        
        color = (64, 255, 0)
        if partA in [8, 9, 10] and partB in [8, 9, 10] and 0 < angle1 < 150:
            color = (0, 64, 255)
            cv2.putText(frame, "Please correct your right leg angle", (50, 75), cv2.FONT_HERSHEY_COMPLEX, .8, (0, 64, 255), 2, lineType=cv2.LINE_AA)
        elif partA in [11, 12, 13] and partB in [11, 12, 13] and 0 < angle2 < 150:
            color = (0, 64, 255)
            cv2.putText(frame, "Please correct your left leg angle", (50, 100), cv2.FONT_HERSHEY_COMPLEX, .8, (0, 64, 255), 2, lineType=cv2.LINE_AA)
        else:
            color = (64, 255, 0)
        
        if key_points[partA] and key_points[partB]:
            cv2.line(frame, key_points[partA], key_points[partB], color, 3, lineType=cv2.LINE_AA)
            cv2.circle(frame, key_points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, key_points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.line(stick_diag, key_points[partA], key_points[partB], color, 3, lineType=cv2.LINE_AA)
            cv2.circle(stick_diag, key_points[partA], 8, (125, 125, 125), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(stick_diag, key_points[partB], 8, (125, 125, 125), thickness=-1, lineType=cv2.FILLED)
    
    cv2.putText(frame, "time taken = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    cv2.putText(stick_diag, "time taken = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    cv2.putText(stick_diag, "Right leg angle = {:.2f} degree".format(angle1), (50, 80), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    cv2.putText(stick_diag, "Left leg angle = {:.2f} degree".format(angle2), (50, 110), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    
    cv2.imshow('Output-Skeleton', frame)
    cv2.imshow('canvas', imutils.resize(stick_diag, width=600))
    orig_video_writer.write(frame)
    stick_diag_video_writer.write(stick_diag)
    
    frame_count += 1
    total_time += (time.time() - t)

orig_video_writer.release()
stick_diag_video_writer.release()
cap.release()
cv2.destroyAllWindows()

print("Total number of frames: ", frame_count)
print("Total time spent: {:0.2f} sec".format(total_time))
print("The total time spent per frame : {:.2f} sec".format(total_time / frame_count))


Error: Could not open video source video1.mp4


SystemExit: 